In [5]:
from openml import tasks

from classes import EAGGA

import os

In [6]:
oml_task_ids = [37, 43, 3903, 3904, 3913, 3918, 10093, 9946, 146819, 359955, 189922, 359962, 190392, 167120, 190137, 190410, 168350, 359975, 359972, 146820]
oml_tasks = tasks.get_tasks(oml_task_ids)

oml_datasets = [oml_task.get_dataset() for oml_task in oml_tasks]

# define positive classes
positive_classes = ['tested_positive', '1', True, True, 'yes', True, '2', '2', '1', '2', '1', True, '1', '1', '2', '1', '2', 'Anomaly', '1', '2']

f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:372: UserWarning: `download_data` will default to False starting in 0.16. Please set `download_data` explicitly to suppress this warning.
  warnings.warn(
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:380: UserWarning: `download_qualities` will default to False starting in 0.16. Please set `download_qualities` explicitly to suppress this warning.
  warnings.warn(


In [41]:
hps = {
    'total_layers': (3, 10),
    'nodes_per_hidden_layer': (3, 20),
    'mu': 15,  # TODO: 100
    'lambda': 5,  # TODO: 10
    'holdout_train_size': 2/3,
    'cv_k': 5
}

batch_size = 16
patience = 10

secs_per_fold = 30
secs_total = 15 * 60

for (oml_dataset, class_positive) in zip(oml_datasets[:1], positive_classes[:1]):  # TODO: remove [:1]
    name = oml_dataset.name
    print(f'Dataset {name}')

    file_path = os.path.join('export', name)
    
    eagga = EAGGA(
        oml_dataset=oml_dataset,
        class_positive=class_positive,
        hps=hps,
        batch_size=batch_size,
        patience=patience,
        secs_per_fold=secs_per_fold,
        secs_total=secs_total,
        file_path=file_path
    )
    eagga.run_eagga()

Dataset diabetes
Starting init population


f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
f:\workplace_github\ws2425-tab-ml\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Finished init population
Start EAGGA at 2025-02-27T22:50:28.666434
Generation 1, evaluate 15 individuals
Running 5-fold CV for individual: 3 total layers, 4 nodes per hidden layer, gs: ([0, 2, 3, 4, 5, 6, 7], [[[1], 1]])
Stop early: 0.6359412317474684 < 0.6667567193508148, epoch stop: 18
Fold 1/5 | trained for 18 epochs | metrics: {'loss': 0.6625715834753854, 'auc': np.float64(0.7725953565505805), 'nf': 0.125, 'ni': 0.0, 'nnm': 0.0}
Stop early: 0.6545042832692464 < 0.6554080347220103, epoch stop: 21
Fold 2/5 | trained for 21 epochs | metrics: {'loss': 0.6368585910115924, 'auc': np.float64(0.8289800995024876), 'nf': 0.125, 'ni': 0.0, 'nnm': 0.0}
Stop early: 0.6634538481632869 < 0.6640152931213379, epoch stop: 47
Fold 3/5 | trained for 47 epochs | metrics: {'loss': 0.6530900767871312, 'auc': np.float64(0.2638888888888889), 'nf': 0.125, 'ni': 0.0, 'nnm': 0.0}
Stop early: 0.6855474104483923 < 0.6965460081895193, epoch stop: 14
Fold 4/5 | trained for 14 epochs | metrics: {'loss': 0.67361588

KeyboardInterrupt: 